<a href="https://colab.research.google.com/github/Aniket-16-S/chatbot/blob/main/chat_bot%20v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import json
import numpy as np
import tensorflow as tf
import random
import nltk
import os # Import os for path manipulation
from nltk.stem import LancasterStemmer
from sklearn.preprocessing import LabelEncoder

stemmer = LancasterStemmer()

# Loading intents from JSON file
with open("faq.json", "r") as file:
    intents = json.load(file)

# Data Preprocessing
words = []
classes = []
documents = []
ignore_words = ["?", "!", ".", ","]
nltk.download('punkt')
# Ensure 'punkt_tab' is downloaded and in a discoverable location
nltk.download('punkt_tab', download_dir=os.path.expanduser('~/nltk_data'))
download_dir = os.path.expanduser('~/nltk_data')
if download_dir not in nltk.data.path:
    nltk.data.path.append(download_dir)


# Tokenizing and stem words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = sorted(set([stemmer.stem(w.lower()) for w in words if w not in ignore_words]))
classes = sorted(set(classes))

# ... (rest of your code remains the same) ...

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [10]:
# Creating the training data

training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = [1 if w in [stemmer.stem(word.lower()) for word in doc[0]] else 0 for w in words]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])


In [11]:
# Convert to numpy arrays
random.shuffle(training)
train_x, train_y = zip(*training)
train_x = np.array(train_x)
train_y = np.array(train_y)


In [12]:
#This creates the model

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_x, train_y, epochs=200, batch_size=8, verbose=0)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0818 - loss: 2.2647
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0693 - loss: 2.3099     
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0943 - loss: 2.2588
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1190 - loss: 2.1864    
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0789 - loss: 2.1859    
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1193 - loss: 2.1638 
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3062 - loss: 2.0146
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1286 - loss: 2.0847    
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2203 - loss: 1.9837
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3478 - loss: 2.0700
Epoch 11/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2687 - loss: 2.0798
Epoch 12/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accu

In [13]:
model.save("chatbot_model.h5")
#saving madel for future use.
# Loading that saved Model
def load_model():
    return tf.keras.models.load_model("chatbot_model.h5")

# Converting user input to a set of words
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return [1 if w in sentence_words else 0 for w in words]


In [14]:
def classify(sentence):
    model = load_model()
    bow = np.array([clean_up_sentence(sentence)])
    prediction = model.predict(bow)[0]
    ERROR_THRESHOLD = 0.6
    results = [[i, p] for i, p in enumerate(prediction) if p > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [(classes[r[0]], r[1]) for r in results]


In [15]:
# This willl Generate response
def chatbot_response(text):
    intents_detected = classify(text)
    if intents_detected:
        intent_tag = intents_detected[0][0]
        for intent in intents["intents"]:
            if intent["tag"] == intent_tag:
                return random.choice(intent["responses"])
    return "I'm sorry, I don't understand."


In [ ]:
print("Chatbot is ready! Type 'exit' to stop.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    print("Chatbot:", chatbot_response(user_input))

Chatbot is ready! Type 'exit' to stop.
You: hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Chatbot: Hi there, how can I help?
You: what are opening timings ?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Chatbot: We're open every day 9am-9pm
You: thank you !


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Chatbot: Any time!
